# Ridge and Lasso

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

%matplotlib inline

In [2]:
# Read the data
data = pd.read_csv('data/bikeshare.csv')

# Year와 Month를 추출
datetime = pd.DatetimeIndex(data['datetime'])
data['year'] = datetime.year
data['month'] = datetime.month
data['hour'] = datetime.hour

# "count" is a method, so it's best to name that column something else
data.rename(columns={'count':'total'}, inplace=True)

# Handling 'season' variable
season_dummies = pd.get_dummies(data.season, prefix='season')
season_dummies.drop(season_dummies.columns[0], axis=1, inplace=True)
data = pd.concat([data, season_dummies], axis=1)

# Add derivative variable "daytime"
data['daytime'] = ((data.hour > 6) & (data.hour < 21)).astype(int)

# Handling 'hour' variable
hour_dummies = pd.get_dummies(data.hour, prefix='hour')
hour_dummies.drop(hour_dummies.columns[0], axis=1, inplace=True)
data = pd.concat([data, hour_dummies], axis=1)

# (참고) 데이터 저장
- 모든 전처리가 끝난 데이터를 한 번 저장해두면 유용하게 사용할 수 있습니다.
- `pandas`에서 제공하는 함수를 사용하여 pickle이나 csv로 저장해봅시다.

In [3]:
## pickle로 저장
data.to_pickle('data/processed_bikeshare.pkl')

In [4]:
## csv로 저장

# data.to_csv('data/processed_bikeshare.csv', index=False, encoding='utf8')
data.to_csv('data/processed_bikeshare.csv', index=False, encoding='cp949')

In [5]:
## pickle로 저장한 데이터 읽기

loaded_data = pd.read_pickle('data/processed_bikeshare.pkl')
print(loaded_data.shape)

(10886, 42)


In [6]:
loaded_data.head(10)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,...,0,0,0,0,0,0,0,0,0,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,...,0,0,0,0,0,0,0,0,0,0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,...,0,0,0,0,0,0,0,0,0,0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,...,0,0,0,0,0,0,0,0,0,0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,...,0,0,0,0,0,0,0,0,0,0
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,...,0,0,0,0,0,0,0,0,0,0
6,2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,2,...,0,0,0,0,0,0,0,0,0,0
7,2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,1,...,0,0,0,0,0,0,0,0,0,0
8,2011-01-01 08:00:00,1,0,0,1,9.84,14.395,75,0.0000,1,...,0,0,0,0,0,0,0,0,0,0
9,2011-01-01 09:00:00,1,0,0,1,13.12,17.425,76,0.0000,8,...,0,0,0,0,0,0,0,0,0,0


In [7]:
## csv로 저장한 데이터 읽기

loaded_data = pd.read_csv('data/processed_bikeshare.csv')
print(loaded_data.shape)

(10886, 42)


In [8]:
loaded_data.head(10)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,...,0,0,0,0,0,0,0,0,0,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,...,0,0,0,0,0,0,0,0,0,0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,...,0,0,0,0,0,0,0,0,0,0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,...,0,0,0,0,0,0,0,0,0,0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,...,0,0,0,0,0,0,0,0,0,0
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,...,0,0,0,0,0,0,0,0,0,0
6,2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,2,...,0,0,0,0,0,0,0,0,0,0
7,2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,1,...,0,0,0,0,0,0,0,0,0,0
8,2011-01-01 08:00:00,1,0,0,1,9.84,14.395,75,0.0000,1,...,0,0,0,0,0,0,0,0,0,0
9,2011-01-01 09:00:00,1,0,0,1,13.12,17.425,76,0.0000,8,...,0,0,0,0,0,0,0,0,0,0


## 선형회귀모델을 학습하는 함수를 조금 수정하였습니다.
다음을 포함하는 dictionary를 출력하는 함수로 변경하였습니다.
- 각 변수에 대응하는 계수들(coefficients)과 intercept
- Train set에서의 RMSE, R^2
- Test set에서의 RMSE, R^2

In [9]:
# Define a function that accepts a list of features and
# returns coefficients, intercept, training RMSE/R^2 and testing RMSE/R^2
def train_test_linreg(d, feature_cols):
    X = d[feature_cols]
    Y = d.total
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
    model = LinearRegression()
    model.fit(X_train, Y_train)
    
    # Make series using selected features and corresponding coefficients
    formula = pd.Series(model.coef_, index = feature_cols)
    
    # Save intercept
    intercept = model.intercept_
    
    # Calculate training RMSE and testing RMSE
    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)
    rmse_train = np.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
    rmse_test = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))
    
    # Calculate training R-square and testing R-square
    rsquared_train = model.score(X_train, Y_train)
    rsquared_test = model.score(X_test, Y_test)
    
    # Make result dictionary
    result={'formula':formula, 'intercept':intercept, 'rmse_train':rmse_train, 'rmse_test':rmse_test,
           'rsquared_train':rsquared_train, 'rsquared_test':rsquared_test}
    
    return result

In [10]:
# Hour에 대한 binary dummy variable만 이용하여 선형회귀모델을 학습
hour_cols = list(data.columns[data.columns.str.startswith('hour_')])
result = train_test_linreg(data, hour_cols)

In [11]:
# Check coefficients, intercept, training RMSE/R^2 and testing RMSE/R^2
result['formula']

hour_1     -22.580917
hour_2     -32.757474
hour_3     -44.209704
hour_4     -49.961957
hour_5     -36.711049
hour_6      16.801172
hour_7     159.656510
hour_8     309.943473
hour_9     160.149618
hour_10    121.193570
hour_11    147.090587
hour_12    202.425259
hour_13    200.110370
hour_14    195.261156
hour_15    195.486156
hour_16    266.890255
hour_17    419.952457
hour_18    379.068371
hour_19    264.786324
hour_20    173.622659
hour_21    125.548656
hour_22     77.631922
hour_23     34.730308
dtype: float64

## Ridge regression & Lasso regression
### 두 모델의 공통점
- **Regularization**: 모델 계수가 커지는 것에 대한 penalty를 부여함으로써 모델의 overfitting(과적합)을 방지
- 기본적인 multiple linear regression (다중선형회귀분석) 은 변수 간의 [다중공선성(multicollinearity)](https://ko.wikipedia.org/wiki/%EB%8B%A4%EC%A4%91%EA%B3%B5%EC%84%A0%EC%84%B1)에 의해 성능이 하락하는데, 이 두 회귀모델은 이에 대해 대처할 수 있는 모델
- 모델의 parameter(모수)가 존재: 계수 크기에 대한 penalty를 얼마나 줄 것인가 (**alpha**)
- alpha가 0이면 단순 다중선형회귀모델과 일치


### Lasso regression의 강점
- Lasso regression은 ridge regression과는 달리 특정 변수의 계수를 0으로 만들어줍니다. 특정 변수의 계수가 0이 아니라는 것은 **lasso regression 모델이 그 변수를 선택**했다고 볼 수 있습니다.
- Lasso regression은 모든 변수가 선택되는 것이 아니라는 점에서 **sparse model** (희소모델)


두 모델을 적용하기에 앞서 다음과 같은 데이터 전처리를 다시 실시하였습니다.
- X에서 가능한 모든 변수를 사용하여, 모델의 성능이 어떻게 나오는지 파악
- 제거한 변수: datetime (수치형 변수가 아니며, year/month/hour로 이미 분리됨),casual & registered (타겟변수인 'total'과 함께 움직이는 변수), total (타겟 변수)

In [12]:
from sklearn.linear_model import Ridge, Lasso

In [13]:
def train_test_ridge(data, alpha_value):
    X = data.drop(['datetime','casual','registered','total', 'season', 'hour'], axis = 1)
    Y = data.total
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
    model = Ridge(alpha = alpha_value)
    model.fit(X_train, Y_train)
    
    # Make series using selected features and corresponding coefficients
    formula = pd.Series(model.coef_, index = list(X.columns.values))
    
    # Save intercept
    intercept = model.intercept_
    
    # Calculate training RMSE and testing RMSE
    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)
    rmse_train = np.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
    rmse_test = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))
    
    # Calculate training R-square and testing R-square
    rsquared_train = model.score(X_train, Y_train)
    rsquared_test = model.score(X_test, Y_test)
    
    # Make result dictionary
    result={'formula':formula, 'intercept':intercept, 'rmse_train':rmse_train, 'rmse_test':rmse_test,
           'rsquared_train':rsquared_train, 'rsquared_test':rsquared_test}
    
    return result

In [14]:
# Ridge regression 학습 및 테스트 (alpha = 0.1)
result = train_test_ridge(data, 1.0)

In [15]:
result['formula']

holiday        -0.196993
workingday      3.594072
weather       -24.418151
temp            2.638343
atemp           2.683150
humidity       -0.720463
windspeed      -0.546934
year           86.691860
month           8.364588
season_2       20.175532
season_3      -16.828603
season_4       -8.516578
daytime       194.466882
hour_1        -20.631902
hour_2        -27.937956
hour_3        -41.159648
hour_4        -44.377989
hour_5        -24.367866
hour_6         30.402766
hour_7        -26.950658
hour_8        124.401884
hour_9        -34.729700
hour_10       -85.080452
hour_11       -63.279141
hour_12       -17.075204
hour_13       -23.455778
hour_14       -36.925508
hour_15       -33.531045
hour_16        34.421169
hour_17       192.728711
hour_18       154.016508
hour_19        48.262827
hour_20       -38.336731
hour_21       106.674313
hour_22        68.617735
hour_23        30.611807
dtype: float64

In [ ]:
def train_test_lasso(data, alpha_value):
    X = data.drop(['datetime','casual','registered','total', 'season', 'hour'], axis = 1)
    Y = data.total
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
    model = Lasso(alpha = alpha_value)
    model.fit(X_train, Y_train)
    
    # Make series using selected features and corresponding coefficients
    formula = pd.Series(model.coef_, index = list(X.columns.values))
    
    # Save intercept
    intercept = model.intercept_
    
    # Calculate training RMSE and testing RMSE
    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)
    rmse_train = np.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
    rmse_test = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))
    
    # Calculate training R-square and testing R-square
    rsquared_train = model.score(X_train, Y_train)
    rsquared_test = model.score(X_test, Y_test)
    
    # Make result dictionary
    result={'formula':formula, 'intercept':intercept, 'rmse_train':rmse_train, 'rmse_test':rmse_test,
           'rsquared_train':rsquared_train, 'rsquared_test':rsquared_test}
    
    return result

In [ ]:
# Lasso regression 학습 및 테스트 (alpha = 0.1)
result = train_test_lasso(data, 1.0)

In [ ]:
result

### 파악할 부분
- Ridge regression과 Lasso regression의 결과와 단순선형회귀모델의 결과를 비교해보세요.
- 위의 Ridge regression과 Lasso regression에서 alpha값을 변형해가면서 결과가 달라지는지 살펴보세요.